In [2]:
import os
import torch
import torchvision
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import wandb

In [3]:
gating_scores = torch.tensor([[0.5,0.6,0.87]])
_, topk_indices = gating_scores.topk(2, dim=1, sorted=False)
torch.zeros_like(gating_scores).scatter_(1, topk_indices, 1)

tensor([[0., 1., 1.]])

In [2]:
device = ("cuda" if torch.cuda.is_available()
          else "cpu")
print(f"Using {device} device !!!")

Using cuda device !!!


In [3]:
output_mode = ["ta", "tc", "ts"]

In [4]:
m = torch.nn.Softmax(dim=1)
input = torch.randn(2, 3)
print(input)
output = m(input)
output

tensor([[ 0.2843, -0.3040, -1.1227],
        [ 0.7276,  0.0094,  0.0984]])


tensor([[0.5555, 0.3085, 0.1360],
        [0.4949, 0.2413, 0.2638]])

In [5]:
torch_input = torch.randn(1,47,requires_grad=False).to('cuda')
torch_input

tensor([[-0.4432,  0.7736,  0.1146,  0.6335, -0.6774,  0.6784,  0.8558,  1.0508,
          0.7136, -1.4981,  0.3902, -0.2166,  0.4850, -0.7565, -0.3012, -1.0641,
         -0.3352, -0.7550,  0.2941,  0.5162,  0.5933, -1.1094, -0.3217,  0.0125,
          0.4780, -0.9503,  0.4873,  0.7150, -0.1319,  0.8677,  0.8407,  0.5326,
          0.0919, -0.3411,  0.2880, -0.7073, -0.0218,  1.2305,  0.3827,  1.6726,
          0.4008, -1.0680, -1.2793, -0.9068,  0.1043,  0.3520,  0.4219]],
       device='cuda:0')

In [6]:
torch_input.shape

torch.Size([1, 47])

In [7]:

model = MoE.MixtureOfExperts(input_dim=47,hidden_dim=128, output_dim=3, num_experts=5, k=2).to('cuda')

In [10]:
loss = torch.nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)

print(input, target)
output = loss(input, target)
output.backward()
print(output)

tensor([[-1.1117, -0.8290, -2.6413, -0.3264, -0.6822],
        [ 0.3927,  2.4550,  0.1521, -0.8914,  1.0866],
        [ 0.9241,  1.1422, -1.7986,  1.1306,  0.0728]], requires_grad=True) tensor([[-1.6564, -0.5222, -0.0663, -1.0255,  1.0090],
        [ 1.5325, -1.3520,  0.3681, -0.9482, -0.9569],
        [-0.4652, -0.2086, -0.6829, -0.2027, -0.3209]])
tensor(2.4881, grad_fn=<MseLossBackward0>)


In [8]:
model(torch_input).tolist()

[[0.6719313263893127]]

In [9]:
torch.onnx.export(model, torch_input, 'MoETransformerModel.onnx', input_names=["features"], output_names=["ThermalVotes"])

### Data

In [4]:
BATCH_SIZE = 32

In [5]:
class ThermalDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.DataFrame):
        self.df = df
        self.features = df[['month', 'day', 'hour', 'minute',
                'room_height', 'room_length', 'room_width', 
                'person_age', 'person_height', 'person_weight', 'person_cloth', 'person_activity',
                'indoor_operative_temperature', 'indoor_mean_radiant_temp', 'indoor_asymmetry_temp', 'indoor_humidity', 'indoor_air_velocity',
                'ashrae_predicted_mean_vote', 'ashrae_predicted_percentage_of_dissatisfied',
                'person_gender_Female', 'person_gender_Male', 
                'season_Summer Season', 'season_Transition Season', 'season_Winter Season',
                'climate_zone_Cold zone',
                'climate_zone_Hot summer and cold winter zone',
                'climate_zone_Severe cold zone', 'building_type _Educational',
                'building_type _Office', 'building_type _Residential',
                'building_function_Bedroom', 'building_function_Classroom',
                'building_function_Living room', 'building_function_Office',
                'building_operation_mode_Air conditioning heating',
                'building_operation_mode_Ceiling capillary heating',
                'building_operation_mode_Cold radiation ceiling cooling',
                'building_operation_mode_Convection cooling',
                'building_operation_mode_Convection heating',
                'building_operation_mode_Naturally Ventilated',
                'building_operation_mode_Others',
                'building_operation_mode_Radiant floor cooling',
                'building_operation_mode_Radiant floor heating',
                'building_operation_mode_Radiator heating']]
        self.thermal_accept = df[["label_thermal_acceptability_vote"]]
        self.thermal_comfort = df[["label_thermal_comfort_vote"]]
        self.thermal_sensation = df[["label_thermal_sensation_vote"]]
        
    def __len__ (self):
        return len(self.df)

    def __getitem__ (self, index:int):
        features = torch.tensor(self.features.loc[index].to_list())
        
        thermal_accept = torch.tensor(self.thermal_accept.loc[index].to_list()[0]).type(torch.LongTensor)
        thermal_comfort = torch.tensor(self.thermal_comfort.loc[index].to_list()[0]).type(torch.LongTensor)
        thermal_sensation = torch.tensor(self.thermal_sensation.loc[index].to_list()[0]).type(torch.LongTensor)
        
        sample = {"features":features, "thermal_accept":thermal_accept, "thermal_comfort":thermal_comfort, "thermal_sensation":thermal_sensation }
        return sample

In [6]:
df = pd.read_csv("Dataset\clean_data\chinese_ver0.1.csv")
df_train, df_valid, df_test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])


train_data = ThermalDataset(df_train.reset_index(drop=True))
valid_data = ThermalDataset(df_valid.reset_index(drop=True))
test_data = ThermalDataset(df_test.reset_index(drop=True))

print(f"Training shape: {df_train.shape} \nValidation shpae: {df_valid.shape} \nTesting shape: {df_test.shape}")

Training shape: (2987, 47) 
Validation shpae: (996, 47) 
Testing shape: (996, 47)


e:\LHK\Study\Lab\SmartCity\3.Others\ThermalComfort\.conda\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
len(train_data)

2987

In [8]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size= BATCH_SIZE)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size= BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size= BATCH_SIZE)

examples = iter(test_dataloader)
example_data = next(examples)

In [9]:
example_data

{'features': tensor([[0.7273, 0.7333, 0.6250,  ..., 0.0000, 0.0000, 0.0000],
         [0.3636, 0.2000, 0.6250,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.4667, 0.4167,  ..., 0.0000, 0.0000, 1.0000],
         ...,
         [1.0000, 0.9667, 0.9167,  ..., 0.0000, 0.0000, 0.0000],
         [1.0000, 0.4667, 0.5000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.4333, 0.7917,  ..., 0.0000, 0.0000, 0.0000]]),
 'thermal_accept': tensor([3, 1, 3, 3, 1, 3, 1, 1, 1, 3, 3, 1, 1, 3, 3, 3, 1, 1, 1, 3, 2, 2, 1, 3,
         1, 1, 3, 3, 3, 3, 3, 1]),
 'thermal_comfort': tensor([0, 1, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
         1, 1, 0, 0, 0, 0, 0, 1]),
 'thermal_sensation': tensor([3, 3, 4, 2, 3, 2, 3, 6, 2, 5, 3, 3, 3, 2, 3, 3, 3, 4, 2, 2, 5, 3, 2, 3,
         3, 3, 4, 4, 3, 3, 3, 2])}

### Model

In [9]:
EPOCH = 10000

### Loss Construction

In [23]:
class ThermalNeuralModel(torch.nn.Module):
    def __init__(self):
        super(ThermalNeuralModel, self).__init__()
        
        # self.net = torchvision.models.resnet18(pretrained=True)
        self.n_features = 44

        self.fc1 = torch.nn.Linear(self.n_features,self.n_features)
        self.relu1 = torch.nn.ReLU()
        self.final1 = torch.nn.Linear(self.n_features, 4)

        self.fc2 = torch.nn.Linear(self.n_features,self.n_features)
        self.relu2 = torch.nn.ReLU()
        self.final2 = torch.nn.Linear(self.n_features, 5)

        self.fc3 = torch.nn.Linear(self.n_features,self.n_features)
        self.relu3 = torch.nn.ReLU()
        self.final3 = torch.nn.Linear(self.n_features, 7)
        
    def forward(self, x):
        out1 = self.fc1(x)
        out1 = self.relu1(out1)
        thermal_accept = self.final1(out1)

        out2 = self.fc2(x)
        out2 = self.relu2(out2)
        thermal_comfort = self.final2(out2)

        out3 = self.fc3(x)
        out3 = self.relu3(out3)
        thermal_sensation = self.final3(out3)

        return thermal_accept, thermal_comfort, thermal_sensation

### Hard Parameter Sharing

In [15]:
class ThermalNeuralModel(torch.nn.Module):
    def __init__(self):
        super(ThermalNeuralModel, self).__init__()
        
        # self.net = torchvision.models.resnet18(pretrained=True)
        self.n_features = 44

        self.inputlayer = torch.nn.Linear(self.n_features,60)
        self.tanh1 = torch.nn.Tanh()
        self.hiddenlayer1 = torch.nn.Linear(60,80)
        self.tanh2 = torch.nn.Tanh()
        self.hiddenlayer2 = torch.nn.Linear(80,100)
        self.tanh3 = torch.nn.Tanh()
        self.hiddenlayer3 = torch.nn.Linear(100,120)
        self.tanh4 = torch.nn.Tanh()
        self.hiddenlayer4 = torch.nn.Linear(120,150)
        self.tanh5 = torch.nn.Tanh()

        self.accept = torch.nn.Linear(150, 4)

        # self.fc2 = torch.nn.Linear(self.n_features,self.n_features)
        # self.relu2 = torch.nn.ReLU()
        self.comfort = torch.nn.Linear(150, 5)

        # self.fc3 = torch.nn.Linear(self.n_features,self.n_features)
        # self.relu3 = torch.nn.ReLU()
        self.sensation = torch.nn.Linear(150, 7)
        
    def forward(self, x):
        out = self.inputlayer(x)

        out = self.tanh1(out)
        out = self.hiddenlayer1(out)

        out = self.tanh2(out)
        out = self.hiddenlayer2(out)

        out = self.tanh3(out)
        out = self.hiddenlayer3(out)

        out = self.tanh4(out)
        out = self.hiddenlayer4(out)

        out = self.tanh5(out)

        thermal_accept = self.accept(out)
        thermal_comfort = self.comfort(out)
        thermal_sensation = self.sensation(out)

        return thermal_accept, thermal_comfort, thermal_sensation

In [13]:
# net = torchvision.models.resnet34(pretrained=True)
# lr = 1e-3
# model = ThermalNeuralModel().to(device=device)

# ts_loss = torch.nn.CrossEntropyLoss()
# tc_loss = torch.nn.CrossEntropyLoss()
# ta_loss = torch.nn.CrossEntropyLoss()

# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.09)


# # writer.add_graph(model, example_data["features"].to(device))

# run = wandb.init(
#     project="Thermal-with-Fully-Chinese",
#     config={
#         'learning_rate': lr,
#         "epochs": EPOCH,
#     }
# )

In [16]:
sweep_config = {
    "method": "random",
    "name": "sweep",
    "metric": {"goal": "minimize", "name": "total_vloss"},
    "parameters": {
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"values": [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000 ]},
        "lr": {"max": 0.01, "min": 0.00001},
    },

}

In [17]:
def train():
    wandb.init(
        project="Thermal-with-Fully-Chinese"
    )
    
    lr = wandb.config.lr
    BATCH_SIZE = wandb.config.batch_size
    EPOCH = wandb.config.epochs
    # total_step = len(train_dataloader)

    model = ThermalNeuralModel().to(device=device)

    ts_loss = torch.nn.CrossEntropyLoss()
    tc_loss = torch.nn.CrossEntropyLoss()
    ta_loss = torch.nn.CrossEntropyLoss()

    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.09)
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size= BATCH_SIZE)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size= BATCH_SIZE)
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size= BATCH_SIZE)

    # writer.add_graph(model, example_data["features"].to(device))

    

    # running_accept_loss = 0.0
    # running_comfort_loss = 0.0
    # running_sensation_loss = 0.0

    # running_accept_correct = 0
    # running_comfort_correct = 0
    # running_sensation_correct = 0

    for epoch in range(EPOCH):
        model.train()
        total_training_loss = 0
        total_valid_loss = 0

        #########################################
        ############ Training process ###########
        #########################################
        running_accept_loss = 0.0
        running_comfort_loss = 0.0
        running_sensation_loss = 0.0

        running_accept_correct = 0
        running_comfort_correct = 0
        running_sensation_correct = 0
        for i, data in enumerate(train_dataloader):
            inputs = data["features"].to(device=device)

            thermal_accept = data["thermal_accept"].to(device=device)
            thermal_comfort = data["thermal_comfort"].to(device=device)
            thermal_sensation = data["thermal_sensation"].to(device=device)

            thermal_accept_output, thermal_comfort_output, thermal_sensation_output  = model(inputs)

            loss_ta = ta_loss(thermal_accept_output, thermal_accept)
            loss_tc = tc_loss(thermal_comfort_output, thermal_comfort)
            loss_ts = ts_loss(thermal_sensation_output, thermal_sensation)

            optimizer.zero_grad()

            total_loss =  loss_ta + loss_tc + loss_ts
            total_loss.backward()
            optimizer.step()

            running_accept_loss += loss_ta.item()
            running_comfort_loss += loss_tc.item()
            running_sensation_loss += loss_ts.item()

            _, thermal_accept_predicted = torch.max(thermal_accept_output,1)
            running_accept_correct += (thermal_accept_predicted == thermal_accept).sum().item()

            _, thermal_comfort_predicted = torch.max(thermal_comfort_output,1)
            running_comfort_correct += (thermal_comfort_predicted == thermal_comfort).sum().item()

            _, thermal_sensation_predicted = torch.max(thermal_sensation_output,1)
            running_sensation_correct += (thermal_sensation_predicted == thermal_sensation).sum().item()

            total_training_loss += total_loss


        #########################################
        ########## Validation process ###########
        #########################################
        valid_accept_loss = 0.0
        valid_comfort_loss = 0.0
        valid_sensation_loss = 0.0

        valid_accept_correct = 0
        valid_comfort_correct = 0
        valid_sensation_correct = 0
        # Set the model to evaluation mode, disabling dropout and using population statistics for batch normalization.
        model.eval()
        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            for i, vdata in enumerate(valid_dataloader):
                inputs = vdata["features"].to(device=device)

                thermal_accept = vdata["thermal_accept"].to(device=device)
                thermal_comfort = vdata["thermal_comfort"].to(device=device)
                thermal_sensation = vdata["thermal_sensation"].to(device=device)

                thermal_accept_output, thermal_comfort_output, thermal_sensation_output  = model(inputs)

                vloss_ta = ta_loss(thermal_accept_output, thermal_accept)
                vloss_tc = tc_loss(thermal_comfort_output, thermal_comfort)
                vloss_ts = ts_loss(thermal_sensation_output, thermal_sensation)

                total_vloss =  vloss_ta + vloss_tc + vloss_ts

                valid_accept_loss += vloss_ta.item()
                valid_comfort_loss += vloss_tc.item()
                valid_sensation_loss += vloss_ts.item()

                _, thermal_accept_predicted = torch.max(thermal_accept_output,1)
                valid_accept_correct += (thermal_accept_predicted == thermal_accept).sum().item()

                _, thermal_comfort_predicted = torch.max(thermal_comfort_output,1)
                valid_comfort_correct += (thermal_comfort_predicted == thermal_comfort).sum().item()

                _, thermal_sensation_predicted = torch.max(thermal_sensation_output,1)
                valid_sensation_correct += (thermal_sensation_predicted == thermal_sensation).sum().item()

                total_valid_loss += total_vloss


            # if(i+1)%10==0:
        print(f"epoch {epoch+1} / {EPOCH}, loss = {total_loss.item():.4f}, valid loss:  {total_vloss.item():.4f}")

        # writer.add_scalar("Thermal Accept Loss", running_accept_loss / len(train_data), epoch)
        # writer.add_scalar("Thermal Comfort Loss", running_comfort_loss / len(train_data), epoch)
        # writer.add_scalar("Thermal Sensation Loss", running_sensation_loss / len(train_data), epoch)

        # writer.add_scalar("Thermal Accept Accuracy", running_accept_correct / len(train_data), epoch)
        # writer.add_scalar("Thermal Comfort Accuracy", running_comfort_correct / len(train_data), epoch)
        # writer.add_scalar("Thermal Sensation Accuracy", running_sensation_correct / len(train_data), epoch)

        wandb.log({"Thermal Accept Loss": running_accept_loss/len(train_data), 
                    "Thermal Comfort Loss": running_comfort_loss/len(train_data), 
                    "Thermal Sensation Loss": running_sensation_loss/len(train_data),
                "Thermal Accept Accuracy": running_accept_correct/len(train_data), 
                "Thermal Comfort Accuracy": running_comfort_correct/len(train_data), 
                "Thermal Sensation Accuracy": running_sensation_correct/len(train_data),

                "Thermal Accept Valid Loss": valid_accept_loss/len(valid_data), 
                    "Thermal Comfort Valid Loss": valid_comfort_loss/len(valid_data), 
                    "Thermal Sensation Valid Loss": valid_sensation_loss/len(valid_data),
                "Thermal Accept Valid Accuracy": valid_accept_correct/len(valid_data), 
                "Thermal Comfort Valid Accuracy": valid_comfort_correct/len(valid_data), 
                "Thermal Sensation Valid Accuracy": valid_sensation_correct/len(valid_data)})
    # torch.save(model.state_dict(),"runs/3_output_v0.4/model/howdoyoufeel.pt")
    # wandb.save("runs/3_output_v0.4/model/howdoyoufeel.pt")

In [18]:
sweep_id = wandb.sweep(sweep=sweep_config, project="Thermal-with-Fully-Chinese")

Create sweep with ID: yb81qiu6
Sweep URL: https://wandb.ai/lhk/Thermal-with-Fully-Chinese/sweeps/yb81qiu6


In [19]:
wandb.agent(sweep_id=sweep_id,function=train, count=2)

wandb: Agent Starting Run: 8at5n7mo with config:
wandb: 	batch_size: 128
wandb: 	epochs: 1000
wandb: 	lr: 0.002785134533219285
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


epoch 1 / 1000, loss = 4.7907, valid loss:  4.7917
epoch 2 / 1000, loss = 4.6371, valid loss:  4.6315
epoch 3 / 1000, loss = 4.4874, valid loss:  4.4693
epoch 4 / 1000, loss = 4.3295, valid loss:  4.2940
epoch 5 / 1000, loss = 4.1677, valid loss:  4.1121
epoch 6 / 1000, loss = 4.0175, valid loss:  3.9414
epoch 7 / 1000, loss = 3.8954, valid loss:  3.8035
epoch 8 / 1000, loss = 3.8075, valid loss:  3.7058
epoch 9 / 1000, loss = 3.7473, valid loss:  3.6417
epoch 10 / 1000, loss = 3.7038, valid loss:  3.5999
epoch 11 / 1000, loss = 3.6702, valid loss:  3.5714
epoch 12 / 1000, loss = 3.6439, valid loss:  3.5510
epoch 13 / 1000, loss = 3.6226, valid loss:  3.5356
epoch 14 / 1000, loss = 3.6049, valid loss:  3.5234
epoch 15 / 1000, loss = 3.5890, valid loss:  3.5132
epoch 16 / 1000, loss = 3.5732, valid loss:  3.5037
epoch 17 / 1000, loss = 3.5613, valid loss:  3.4960
epoch 18 / 1000, loss = 3.5525, valid loss:  3.4902
epoch 19 / 1000, loss = 3.5448, valid loss:  3.4850
epoch 20 / 1000, loss

Thermal Accept Accuracy,▁▁▃▃▃▃▄▄▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▆▆▇▇▇▇▇▇▇█▇█▇█
Thermal Accept Loss,█▆▆▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁
Thermal Accept Valid Accuracy,▁▃▃▂▃▃▃▃▅▅▅▆▇▆▇▆▆█▇▇▇▇█▇▇█▇▆██▆█▇▇▆▇▇▆▆▆
Thermal Accept Valid Loss,█▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▂▂▁▂▂▂▂▂▁▁▁▂▁▁▁▂▂
Thermal Comfort Accuracy,▅▅▅▃▁▂▁▁▁▂▃▅▅▆▆▆▆▅▆▅▆▆▆▆▇▇▇▆▇▇▇▇▆▇▇▇▇█▇█
Thermal Comfort Loss,█▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Thermal Comfort Valid Accuracy,▆▆▆▂▁▁▁▁▁▁▂▆▆▆▆▅▆▆▅▆▆▆▇█▇▇▆▆▆▇▇▇▇▆▇▇▇▇▇▇
Thermal Comfort Valid Loss,█▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▂▁▂▂▁▁▂▁▁▁▁▁▁▁▂▁
Thermal Sensation Accuracy,▂▂▂▂▂▃▂▁▁▂▁▂▃▂▂▄▄▅▅▆▆▄▃▄▄▆▆█▅▄▅▄▃▆▄▅▆▅▅▇
Thermal Sensation Loss,█▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Thermal Sensation Valid Accuracy,▄▄▄▄▅███████▇███▇▆▅▄▄▄▄▄▄▄▄▄▅▄▄▄▄▅▅▅▄▃▁▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: cvttr8jr with config:
wandb: 	batch_size: 64
wandb: 	epochs: 1000
wandb: 	lr: 0.001784570990624434
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


epoch 1 / 1000, loss = 4.6966, valid loss:  4.7096
epoch 2 / 1000, loss = 4.4191, valid loss:  4.4012
epoch 3 / 1000, loss = 4.1545, valid loss:  4.1009
epoch 4 / 1000, loss = 3.9391, valid loss:  3.8577
epoch 5 / 1000, loss = 3.8106, valid loss:  3.7121
epoch 6 / 1000, loss = 3.7342, valid loss:  3.6340
epoch 7 / 1000, loss = 3.6817, valid loss:  3.5914
epoch 8 / 1000, loss = 3.6426, valid loss:  3.5655
epoch 9 / 1000, loss = 3.6139, valid loss:  3.5492
epoch 10 / 1000, loss = 3.5915, valid loss:  3.5385
epoch 11 / 1000, loss = 3.5666, valid loss:  3.5298
epoch 12 / 1000, loss = 3.5494, valid loss:  3.5248
epoch 13 / 1000, loss = 3.5342, valid loss:  3.5206
epoch 14 / 1000, loss = 3.5253, valid loss:  3.5182
epoch 15 / 1000, loss = 3.5164, valid loss:  3.5161
epoch 16 / 1000, loss = 3.5030, valid loss:  3.5131
epoch 17 / 1000, loss = 3.4935, valid loss:  3.5098
epoch 18 / 1000, loss = 3.4845, valid loss:  3.5090
epoch 19 / 1000, loss = 3.4787, valid loss:  3.5079
epoch 20 / 1000, loss

Thermal Accept Accuracy,▁▃▃▃▃▃▄▄▅▆▆▆▆▇▆▆▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████
Thermal Accept Loss,█▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
Thermal Accept Valid Accuracy,▁▃▃▃▃▃▃▄▆▆▆▆▆▇▇▇▇▇▆▆▇▇███▇███████████▇█▇
Thermal Accept Valid Loss,█▇▆▆▅▅▅▄▄▄▃▃▃▃▃▂▃▂▃▃▂▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▂▂▂
Thermal Comfort Accuracy,▅▅▃▁▁▁▃▄▅▅▆▅▆▅▆▆▆▇▇▇▇▇▅▇▆▇██▇▇▆█████▇▇▇█
Thermal Comfort Loss,█▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁
Thermal Comfort Valid Accuracy,██▃▁▂▂▃▇█▇▅▅▄▆▅▅▆▄▄▃▄▆▆▃▅▅▆▅▅▅▆▆▆▆▆▆▅▆█▇
Thermal Comfort Valid Loss,█▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▁
Thermal Sensation Accuracy,▁▁▁▁▃▃▂▂▂▃▃▃▁▁▂▄▅▂▃▃▄▇█▆▆▅▅▅▆▄▅▄▃▃▅▄▄▅▅█
Thermal Sensation Loss,█▆▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
Thermal Sensation Valid Accuracy,▃▃▃▃▅▇█▇▇▇▇▇████████▇▅▅▆▆▆▆▆▆▆▅▅▆▅▅▃▃▁▁▁


### Evaluate

In [27]:
with torch.no_grad():
    n_accept_correct = 0
    n_comfort_correct = 0
    n_sensation_correct = 0
    for index, data in enumerate(test_dataloader):
        inputs = data["features"].to(device=device)

        thermal_accept = data["thermal_accept"].to(device=device)
        thermal_comfort = data["thermal_comfort"].to(device=device)
        thermal_sensation = data["thermal_sensation"].to(device=device)

        thermal_accept_output, thermal_comfort_output, thermal_sensation_output  = model(inputs)

        _, thermal_accept_predicted = torch.max(thermal_accept_output,1)
        n_accept_correct += (thermal_accept_predicted == thermal_accept).sum().item()

        _, thermal_comfort_predicted = torch.max(thermal_comfort_output,1)
        n_comfort_correct += (thermal_comfort_predicted == thermal_comfort).sum().item()

        _, thermal_sensation_predicted = torch.max(thermal_sensation_output,1)
        n_sensation_correct += (thermal_sensation_predicted == thermal_sensation).sum().item()


accuracy_thermal_accept = 100*(n_accept_correct/len(test_data))
accuracy_thermal_comfort = 100*(n_comfort_correct/len(test_data))
accuracy_thermal_sensation = 100*(n_sensation_correct/len(test_data))


print(f"----------ACCURACYYYYYYYY----------\nThermal Accept: {accuracy_thermal_accept:.4f}% \nThermal Comfort: {accuracy_thermal_comfort:.4f}% \nThermal Sensation: {accuracy_thermal_sensation:.4f}%")

----------ACCURACYYYYYYYY----------
Thermal Accept: 64.9598% 
Thermal Comfort: 59.1365% 
Thermal Sensation: 45.6827%


### Export the model to ONNX Format

In [24]:
torch.save(model.state_dict(),"runs/3_output_v0.4/model/howdoyoufeel.pt")

In [15]:
import onnx
import onnxruntime

In [26]:
model = ThermalNeuralModel().to(device=device)
model.load_state_dict(torch.load("runs/3_output_v0.3/model/howdoyoufeel.pt"))

<All keys matched successfully>

In [73]:
for index, data in enumerate(test_dataloader):
        inputs = data["features"].to(device=device)
        print(inputs.shape)
        break

torch.Size([32, 44])


In [17]:
torch_input = torch.randn(1,1,1,44,requires_grad=False).to('cuda')
torch_output = model(torch_input)
onnx_program = torch.onnx.export(model, torch_input, "runs/3_output_v0.4/model/howdoyoufeel.onnx",
                                 export_params=True,
                                 opset_version=10,
                                 do_constant_folding=True,
                                 input_names=['input'],
                                 output_names=['thermal_accept', 'thermal_comfort', 'thermal_sensation'],
                                 dynamic_axes={'input':{0:"BATCH_SIZE"},
                                               'output':{0:'BATCH_SIZE'}})

e:\LHK\Study\Lab\SmartCity\3.Others\ThermalComfort\.conda\lib\site-packages\torch\onnx\utils.py:2095: UserWarning: Provided key output for dynamic axes is not a valid input/output name
  warnings.warn(


In [75]:
onnx_model = onnx.load("runs/3_output_v0.2/model/howdoyoufeel.onnx")
onnx.checker.check_model(onnx_model)

In [85]:
torch_input

tensor([[[[ 0.4826, -0.7066,  0.5693, -0.3838,  0.1825, -0.4439,  0.9115,
           -1.5038,  1.6719, -0.6861,  1.5177, -0.5249,  2.2697, -1.3663,
            0.3071, -0.5139, -1.0905, -1.2862, -0.8850, -0.1891, -0.2712,
            0.3541, -1.1853, -1.1704,  0.8803,  0.1760,  0.4272, -1.1351,
           -1.3588,  0.8517, -0.3747,  1.2604,  0.3989,  0.8105,  0.1295,
            0.5229,  0.5804, -0.1092, -0.1623,  0.3918, -0.6512, -1.5955,
           -1.1637, -1.0300]]]], device='cuda:0')

In [81]:
ort_session = onnxruntime.InferenceSession("runs/3_output_v0.2/model/howdoyoufeel.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.cpu().detach().numpy() 

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(torch_input)}
ort_outs = ort_session.run(None, ort_inputs)

print(f"Output from Torch model: \n{torch_output}\n----------------------\nOutput from ONNX model: \n{ort_outs}")

Output from Torch model: 
(tensor([[[[-1.2250,  0.3207, -3.1234,  2.3189]]]], device='cuda:0',
       grad_fn=<ViewBackward0>), tensor([[[[ 0.8645,  0.8995,  0.1781, -0.3676,  0.3345]]]], device='cuda:0',
       grad_fn=<ViewBackward0>), tensor([[[[-0.3605, -0.0172, -0.6571,  0.6817,  0.0479,  0.1444,  0.1339]]]],
       device='cuda:0', grad_fn=<ViewBackward0>))
----------------------
Output from ONNX model: 
[array([[[[-1.2249525 ,  0.32065603, -3.1233754 ,  2.3188586 ]]]],
      dtype=float32), array([[[[ 0.86445314,  0.8995396 ,  0.17808047, -0.36761838,
           0.33453998]]]], dtype=float32), array([[[[-0.36052948, -0.01719668, -0.657147  ,  0.68170255,
           0.04792242,  0.14438152,  0.13392255]]]], dtype=float32)]
